<a href="https://colab.research.google.com/github/zhiyuan-95/portfolio_optimizer/blob/master/portfolio_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pandas_datareader import data as pdr
import yfinance as yfin
from datetime import datetime, timedelta,date
import time
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [7]:
yfin.pdr_override()

In [220]:
class portfolio:
  def __init__(self,tickers, risk_free_rate = 0.55):
    self._tickers = tickers
    self._div_history = pd.DataFrame()
    self._price_history = pd.DataFrame()
    self.get_stocks()
    self._t0 = ''
    self._period = 12
    self._t1 = ''
    self._price= pd.DataFrame()
    self._return = []
    self._div = pd.DataFrame()
    self._corr = 'numpy'
    self._mu = []
    self._risk_free_rate = risk_free_rate
  def get_stocks(self):
      file_names = []
      temp = []
      div_date = []
      div_history = {}
      iter = 0
      for x in self._tickers:
        Start = '2000-1-1'
        End = '2023-1-1'
        self._price_history[x] = pdr.get_data_yahoo(x,start=Start, end=End)['Adj Close']
        div_history[x] = yfin.Ticker(x).dividends
        div_date+=list(div_history[x].index)
      div_date = sorted(div_date)
      self._div_history.index= map(lambda x: x.strftime('%Y-%m-%d'), div_date)
      for y in self._tickers:
        self._div_history[y] =div_history[y]
        self._div_history[y] = self._div_history[y].fillna(0)
  def update(self, date):
    self._t1 = datetime.strptime(date,'%Y-%m-%d')
    self._t0 = self._t1-timedelta(weeks = self._period)
    start = self._t0.strftime('%Y-%m-%d')
    end = self._t1.strftime("%Y-%m-%d")
    last_year = str(int(end[:4])-1)
    self._price = self._price_history[start:end]
    self._div = self._div_history[self._div_history.index.str.startswith(last_year)].to_numpy()
    self._corr = self._price.corr().to_numpy()
    self._price = self._price.to_numpy()
    self.Expected_return()
  def Return(self):
    # return  = return on stock value from t0-t1 + expected on dividend
    # expected return on dividend of period t of year n = sum of dividend of year n-1 *(t/52)
    p0 = self._price[0]
    p1 = self._price[1]
    return_p = (p1-p0)/p0
    div_rate = sum(self._div)/p0
    E_return_d = div_rate*self._period/52
    return return_p+E_return_d
  def pastReturn(self,years=1):
    rtn = np.empty((years,len(tickers)))
    for t in range(years):
      the_year = str(int(self._t1.strftime("%Y-%m-%d")[:4])-t-1)
      stock_last_yr= self._price_history[self._price_history.index.strftime("%Y-%m-%d").str.startswith(the_year)].to_numpy()
      div_sum = self._div_history[self._div_history.index.str.startswith(the_year)].to_numpy()
      rtn[t] = (sum(div_sum)+stock_last_yr[-1]-stock_last_yr[0])/stock_last_yr[0]
    return sum(rtn)/3
  def Expected_return(self,w = 1):
    self._mu = w*self.Return()+(1-w)*self.pastReturn()
  def negative_sharpe(self, weights):
    portfolio_return = np.dot(weights, self._mu)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(self._corr, weights)))
    sharpe_ratio = (portfolio_return - self._risk_free_rate) / portfolio_stddev
    return -sharpe_ratio
  def optimized_weights(self):
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    initial_weights = np.ones(len(self._mu)) / len(self._mu)
    # Perform the optimization
    bounds = tuple((0, 1) for _ in range(len(self._mu)))
    result = minimize(self.negative_sharpe, initial_weights,bounds=bounds, constraints=constraints)
    # Extract the optimized weights
    optimized_weights = result.x
    return np.round(optimized_weights, decimals = 2)

In [221]:
tickers = ['qqq','dia','spy','iwm']
p = portfolio(tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [222]:
date = '2003-01-03'
for x in range(250):
  p.update(date)
  weight = p.optimized_weights()
  date= datetime.strptime(date,'%Y-%m-%d')
  t1 = date+timedelta(weeks = 4)
  date = t1.strftime('%Y-%m-%d')
  print(date,weight)

2003-01-31 [0. 0. 1. 0.]
2003-02-28 [0.   0.48 0.52 0.  ]
2003-03-28 [0. 1. 0. 0.]
2003-04-25 [0. 0. 1. 0.]
2003-05-23 [0.   0.45 0.55 0.  ]
2003-06-20 [0. 0. 1. 0.]
2003-07-18 [0.   0.39 0.61 0.  ]
2003-08-15 [0. 1. 0. 0.]
2003-09-12 [1. 0. 0. 0.]
2003-10-10 [0. 0. 1. 0.]
2003-11-07 [0. 1. 0. 0.]
2003-12-05 [1. 0. 0. 0.]
2004-01-02 [0.   0.49 0.51 0.  ]
2004-01-30 [0.   0.27 0.73 0.  ]
2004-02-27 [0. 0. 1. 0.]
2004-03-26 [0.   0.42 0.58 0.  ]
2004-04-23 [0.  0.5 0.5 0. ]
2004-05-21 [0. 0. 1. 0.]
2004-06-18 [0. 0. 1. 0.]
2004-07-16 [0. 0. 1. 0.]
2004-08-13 [0. 0. 1. 0.]
2004-09-10 [0. 0. 1. 0.]
2004-10-08 [0. 0. 1. 0.]
2004-11-05 [0. 0. 1. 0.]
2004-12-03 [0. 0. 0. 1.]
2004-12-31 [0. 0. 1. 0.]
2005-01-28 [0.  0.4 0.6 0. ]
2005-02-25 [0. 0. 1. 0.]
2005-03-25 [0. 0. 1. 0.]
2005-04-22 [0. 0. 1. 0.]
2005-05-20 [0. 0. 1. 0.]
2005-06-17 [0. 0. 1. 0.]
2005-07-15 [0. 0. 1. 0.]
2005-08-12 [0. 0. 1. 0.]
2005-09-09 [0. 0. 1. 0.]
2005-10-07 [0. 0. 1. 0.]
2005-11-04 [0. 0. 1. 0.]
2005-12-02 [0. 0. 1